In [59]:
%pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma gradio python-dotenv
%pip install langchain_cohere

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [60]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the Google Drive path where your documents are stored
# google_drive_path = '/content/drive/MyDrive/local_rag/data'

In [61]:
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"
import os

# import the .env file
from dotenv import load_dotenv
load_dotenv()
# print(os.environ.get("COHERE_API_KEY")) 

True

In [62]:
from langchain_chroma import Chroma
import gradio as gr
from langchain.embeddings import SentenceTransformerEmbeddings

# Load the embedding model
embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Use the model with LangChain as before
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)



In [63]:
num_results = 5

retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

docs = retriever.invoke("What is attention")
print(docs)

docs = retriever.invoke("What is communication")
print(docs)

[Document(id='98369a59-fdd6-4571-8f37-2c076971e0b0', metadata={'creationdate': '', 'creator': 'Google', 'page': 16, 'page_label': '17', 'producer': 'PyPDF', 'source': 'data/M11.1__Introduction_to_Machine_Learning.pdf', 'title': 'M11.1: Introduction to Machine Learning', 'total_pages': 74}, page_content='2. It measures how closely that action \ntook it to completing a task.\n1. The agent observes \nits environment and \ntakes an action.\nEnvironment\nAgent\nState\nReward\nAction'), Document(id='1843a1d7-1aeb-48e4-b87f-3287cfe3d0cb', metadata={'creationdate': '', 'creator': 'Google', 'page': 18, 'page_label': '19', 'producer': 'PyPDF', 'source': 'data/M1.2_ The Impact of Machine Learning.pdf', 'title': 'M1.2: The Impact of Machine Learning', 'total_pages': 50}, page_content='— Merritt, R. 2022. What is a transformer model? [Blog, 25 March].\nAvailable: https://blogs.nvidia.com/blog/2022/03/25/what-is-a-transformer-model/ [2023, April 5].\n“A transformer model is a neural network that \nl

In [64]:
docs = retriever.invoke("What is attention")
print(docs)
knowledge = ""
sources = []
for doc in docs:
    knowledge += doc.page_content+"\n\n"
    print(doc.page_content)

for doc in docs:
    sources.append({"source":doc.metadata["source"], "page_label":doc.metadata["page_label"]})

print(sources)

[Document(id='98369a59-fdd6-4571-8f37-2c076971e0b0', metadata={'creationdate': '', 'creator': 'Google', 'page': 16, 'page_label': '17', 'producer': 'PyPDF', 'source': 'data/M11.1__Introduction_to_Machine_Learning.pdf', 'title': 'M11.1: Introduction to Machine Learning', 'total_pages': 74}, page_content='2. It measures how closely that action \ntook it to completing a task.\n1. The agent observes \nits environment and \ntakes an action.\nEnvironment\nAgent\nState\nReward\nAction'), Document(id='1843a1d7-1aeb-48e4-b87f-3287cfe3d0cb', metadata={'creationdate': '', 'creator': 'Google', 'page': 18, 'page_label': '19', 'producer': 'PyPDF', 'source': 'data/M1.2_ The Impact of Machine Learning.pdf', 'title': 'M1.2: The Impact of Machine Learning', 'total_pages': 50}, page_content='— Merritt, R. 2022. What is a transformer model? [Blog, 25 March].\nAvailable: https://blogs.nvidia.com/blog/2022/03/25/what-is-a-transformer-model/ [2023, April 5].\n“A transformer model is a neural network that \nl

In [65]:
from langchain.chat_models import ChatCohere
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Initialize Cohere LLM
llm = ChatCohere(model="command-r-plus-08-2024")

# Load the embedding model
embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Initialize Vector Store
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)

# Set up retriever
num_results = 5
retriever = vector_store.as_retriever(search_kwargs={'k': num_results})


In [66]:

def stream_response(message, history, skipSources=False):
    """Handles chatbot conversation in Gradio."""
    partial_message = ""
    # Convert Gradio history to LangChain format
    chat_history = [SystemMessage(content="You are a helpful assistant. Answer based on the provided knowledge.")]
    for user_msg, bot_msg in history:
        chat_history.append(HumanMessage(content=user_msg))
        chat_history.append(AIMessage(content=bot_msg))

    # Retrieve relevant documents
    docs = retriever.invoke(message)

    # Format retrieved knowledge
    knowledge = "\n\n".join([doc.page_content for doc in docs])
    sources = "\n".join([f"- **Document**: {doc.metadata['source']}, **Page**: {doc.metadata.get('page_label', 'N/A')}" for doc in docs])
    

    # Construct the prompt
    rag_prompt = f"""
    You are an assistant that answers questions based solely on the provided knowledge.
    If the knowledge does not contain the answer, say you don't know.
    
    The knowledge:
    {knowledge}
    """

    # Append user query
    chat_history.append(HumanMessage(content=message))
    chat_history.append(AIMessage(content=rag_prompt))


    last_chunk = None  # Track last chunk
    for response in llm.stream(chat_history):
        partial_message += response.content
        last_chunk = response.content  # Keep track of the final chunk
        yield partial_message  # Send intermediate responses

    # Append sources only to the last streamed chunk
    if last_chunk:
        uncertainty_phrases = ["I don't know", "I'm not sure if I can answer that", "I'm sorry"]
        if not skipSources and not any(phrase.lower() in partial_message.lower() for phrase in uncertainty_phrases):
            final_response = partial_message + "\n\n"+ f"Sources:\n" + sources 
        else:
            final_response = partial_message 
        yield final_response 
        


In [67]:
import json

with open('test_cases.json', 'r') as file:
    test_cases = json.load(file)

test_cases


[{'query': 'What is machine learning?',
  'expected': 'Machine learning is a field within AI that develops algorithms and models enabling computers to learn and make decisions without explicit programming. It allows machines to analyze data, identify patterns, and make informed decisions with minimal human intervention by training on vast datasets. ML is categorized into three types: Supervised Learning, where algorithms learn from labeled data to make predictions; Unsupervised Learning, where algorithms find patterns in unlabeled data for clustering and anomaly detection; and Reinforcement Learning, where models learn through rewards and penalties to optimize decision-making. Applications include image and speech recognition, NLP, recommendation systems, fraud detection, predictive analytics, and autonomous driving, making ML a key part of modern AI systems.',
  'generated': "I don't know."},
 {'query': 'What is logistic regression?',
  'expected': 'Logistic regression is a statistica

In [68]:

# Generate responses for each test case
for test_case in test_cases:
    response_generator = stream_response(test_case["query"], [], True)
    generated_response = ""
    for partial_response in response_generator:
        generated_response = partial_response  # Get the final response
    test_case["generated"] = generated_response

# Save the test cases to a JSON file
with open("test_cases.json", "w") as file:
    json.dump(test_cases, file, indent=4)

# Print the test cases
print(json.dumps(test_cases, indent=4))

[
    {
        "query": "What is machine learning?",
        "expected": "Machine learning is a field within AI that develops algorithms and models enabling computers to learn and make decisions without explicit programming. It allows machines to analyze data, identify patterns, and make informed decisions with minimal human intervention by training on vast datasets. ML is categorized into three types: Supervised Learning, where algorithms learn from labeled data to make predictions; Unsupervised Learning, where algorithms find patterns in unlabeled data for clustering and anomaly detection; and Reinforcement Learning, where models learn through rewards and penalties to optimize decision-making. Applications include image and speech recognition, NLP, recommendation systems, fraud detection, predictive analytics, and autonomous driving, making ML a key part of modern AI systems.",
        "generated": "Machine learning is a branch of artificial intelligence (AI) that enables computer s

In [69]:
import gradio as gr

with gr.Blocks(css="""
    body {background-color: #121212; color: #e0e0e0;}
    .gradio-container {background-color: #121212;}
    .chat-message {border-radius: 10px; padding: 10px; margin: 5px 0; font-size: 20px;}  /* Increased font size */
    .chat-message.user {background-color: #1e1e1e; color: #ffffff; text-align: right; font-size: 25px;} /* Bigger font for user */
    .chat-message.bot {background-color: #333333; color: #bb86fc; font-weight: bold; font-size: 25px;} /* Bigger font for bot */
""") as demo:
    
    gr.Markdown(
        "<h1 style='text-align: center; color: #bb86fc;'>💬 Docu-Chat</h2>",
    )

    chatbot = gr.ChatInterface(
        stream_response,
        chatbot=gr.Chatbot(
            bubble_full_width=False,
            avatar_images=("./user.png", "./bot.png"),  # Change avatar images
        ),
        textbox=gr.Textbox(
            placeholder="Type your message...",
            container=False,
            autoscroll=True,
            
        ),
    )

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7879


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://01c716bde7e8ee4ab4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
